## Model validation

Here we validate our models on the simulated data they generated.

In [1]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import scipy
import scipy.stats
import networkx as nx
import pandas as pd
import pickle
from MonsterPrior import MonsterPrior
from importlib import reload
from pathlib import Path
from SuccessorRepresentation import SuccessorRepresentation
from GraphGP import LaplacianGP
import copy
import time
from utils import *
%matplotlib inline


In [2]:
### In this script I validate each model on the data generated by every model we consider

def add_diff(matrix, predictions, trial):

    for i, pred in enumerate(predictions):
        diff = pred[0] - pred[1]
        matrix[trial, i] = diff

def validate_model_sims(choice_data, simulation, simulation_name, params):

    """ Here we make each model go through a dataset generated by another model and generate predictions on monster values based on the rewards and decisions made by that model. The goal is to see if we can recover the ground truth models in the first place"""
    
    op1 = np.array(choice_data["option1"])
    op2 = np.array(choice_data["option2"])
    op1 -= 1
    op2 -= 1
    contexts = np.array(choice_data["map"])
    subj = np.array(choice_data["subj"])
    last_subj = -1
    subj_counter = -1
    choices = np.array(simulation["choices"], dtype=int)
    decisions = np.array(simulation["choice_indices"])
    rewards = np.array(simulation["rewards"])
    states = np.arange(12)
    sr_diffusion = 1
    
    header = ["SR-GP", "Euclidean", "Compositional", "Mean-tracker", "Optimized-SR-GP"]
    predicted_differences = np.zeros((len(subj), len(header)))

    lengthscale = params["lengthscale"]
    comp_lengthscale = params["compositional_lengthscale"]
    learning_rate = params["learning_rate"]
    comp_lr = params["compositional_lr"]
    
    for i, subj_id in enumerate(subj):
        current_context = contexts[i]
        if subj_id != last_subj:


            subj_counter += 1
            trial_counter = 0

            loc = PI_dict[subj_id]

            context_dict = {}
            context_dict[1] ={"training_idx": [], "rewards": [], "state_rewards" : np.zeros(len(np.arange(12)))} #copy.deepcopy(dict_template)
            context_dict[2] = {"training_idx": [], "rewards": [], "state_rewards" : np.zeros(len(np.arange(12)))} #copy.deepcopy(dict_template)

            ### SR

            seq_list = []
            for run, seq in transition_dict[subj_id].items():
                seq_ = copy.deepcopy(seq)
                seq_ -=1
                seq_list.append(seq_)

            sr_model = SuccessorRepresentation(states, seq_list, alpha=learning_rate)
            SR = sr_model.get_SR()

            SRL = estimate_laplacian(SR, gamma = sr_model.gamma, subj_id = subj_id, plot=False)

            SR_kernel = scipy.linalg.expm(-sr_diffusion*SRL)
            ## sr for the compositional kernel:
            sr_model = SuccessorRepresentation(states, seq_list, alpha=comp_lr)
            SR = sr_model.get_SR()

            SRL = estimate_laplacian(SR, gamma = sr_model.gamma, subj_id = subj_id, plot=False)
            SR_kernel_comp = scipy.linalg.expm(-sr_diffusion*SRL)
            ### Euclidean

            estimated_euclidean_kernel = RBF(loc, loc, l=lengthscale)
            estimated_euclidean_kernel_comp = RBF(loc, loc, l=comp_lengthscale)

            comp_kernel = (estimated_euclidean_kernel_comp + SR_kernel_comp)/2

            ### add observations for this context
            options = [op1[i], op2[i]]
            choice = choices[i]
            reward = rewards[i]


            context_dict[current_context]["training_idx"].append(choice)
            context_dict[current_context]["rewards"].append(reward)



            ## set the last subj_id to the current one
            last_subj = subj_id
            trial_counter += 1


        elif len(context_dict[current_context]["rewards"]) == 0:  # check if participant has been able to make any observations in this context yet 
            # if not then let choice be random, and store observations into context dict

            options = [op1[i], op2[i]]
            choice = choices[i]
            reward = rewards[i]

            context_dict[current_context]["training_idx"].append(choice)
            context_dict[current_context]["rewards"].append(reward)

            trial_counter += 1


        else:
            options = [op1[i], op2[i]]

            choice = choices[i]
            decision = decisions[i]
            unchosen = 1 - decision
            reward = rewards[i]

            training_idx = context_dict[current_context]["training_idx"] # the training indices for the gps
            R = copy.copy(context_dict[current_context]["state_rewards"]) # an array with rewards for each state for the SR. We copy so that it doesn't change when we normalize it
            y = np.array(copy.copy(context_dict[current_context]["rewards"]))  # for use in the gp models. we copy this so we can normalize it and convert it into an array without messing with the original set of reward observations
            ## we don't normalize now

            y_prime = np.append(y, reward)

            if y.std() != 0:
                y = (y- y.mean())/y.std()
                y_prime = (y_prime - y_prime.mean())/y_prime.std()

            else:
                y = (y - y.mean())
                y_prime = (y_prime - y_prime.mean())

    
            ### Euclidean prediction error
            SR_GP_preds = estimate_GP(SR_kernel, y, training_idx, option_indices=options)
            euclidean_preds = estimate_GP(estimated_euclidean_kernel, y, training_idx, option_indices=options)
            comp_preds = estimate_GP(comp_kernel, y, training_idx, option_indices=options)
            MT_preds = estimate_GP(np.eye(12), y, training_idx, option_indices=options)
            optimized_sr_gp_preds = optimize_diffusion_gp(SRL, training_idx, y, option_indices = options)

            model_predictions = [SR_GP_preds, euclidean_preds, comp_preds, MT_preds, optimized_sr_gp_preds]



            add_diff(matrix=predicted_differences, predictions=model_predictions, trial=i)
            

            ### update arrays:
            context_dict[current_context]["training_idx"].append(choice)
            context_dict[current_context]["rewards"].append(reward)


            trial_counter += 1

    diff_df = pd.DataFrame(predicted_differences)
    diff_df.to_csv(f"model_simulations/validations/{simulation_name}-validation.csv", index=False, header = header)



In [3]:
### load simulated data

choice_data = pd.read_csv("choice_data.csv")
folder = "model_simulations/simulations"
sr_gp_sim = pd.read_csv(f"{folder}/sr_gp_sim.csv")
euc_sim = pd.read_csv(f"{folder}/euclidean_sim.csv")
comp_sim = pd.read_csv(f"{folder}/compositional_sim.csv")
mt_sim =  pd.read_csv(f"{folder}/mean_tracker_sim.csv")
sr_gp_opt_sim =  pd.read_csv(f"{folder}/sr_gp_sim_optimized.csv")

### open pickled data
with open('occupancy_counts.pickle', 'rb') as handle:
    occupancy_dict = pickle.load(handle)

with open('transitions.pickle', 'rb') as handle:
    transition_dict = pickle.load(handle)

with open('subjective_kernel.pickle', 'rb') as handle:
    subjective_kernel_dict = pickle.load(handle)

with open('subjective_grid_search_dict.pickle', 'rb') as handle:
    subj_kernel_grid_dict = pickle.load(handle)


with open('path_integration_kernels.pickle', 'rb') as handle:
    estimated_euclidean_kernels = pickle.load(handle)

with open('path_integration_monster_locations.pickle', 'rb') as handle:
    PI_dict = pickle.load(handle)



In [5]:
### run models on simulated data

params = {"lengthscale": 1.24, "learning_rate": 0.4125, "compositional_lengthscale": 2.05, "compositional_lr": 0.01}
simulation_datasets = [sr_gp_sim, euc_sim, comp_sim, mt_sim, sr_gp_opt_sim]
sim_names = ["SR-GP", "Euclidean", "Compositional", "Mean-tracker", "SR-GP-optimized"]



for (dataset, name) in zip(simulation_datasets, sim_names):
    validate_model_sims(choice_data, simulation=dataset, simulation_name=name, params=params)